In [ ]:
from larcv import larcv
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
larcv.load_pyutil
larcv.load_cvutil
import cv2
import ROOT
from ROOT import fcllite
from ROOT import geo2d
pdraw = geo2d.PyDraw()

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
pprocessor = larcv.PreProcessor()
print pprocessor

In [ ]:
ctor = ROOT.std.vector(ROOT.cv.Point_('int'))()

In [ ]:
# make a fake TRACK image...
a=np.zeros((485,485))

trk_ctor = ROOT.std.vector(ROOT.cv.Point_('int'))()

trk_ctor.clear()
    
pt_x=[]
pt_y=[]

trk_ctor.push_back(ROOT.cv.Point_("int")(25,225))
trk_ctor.push_back(ROOT.cv.Point_("int")(50,225))
trk_ctor.push_back(ROOT.cv.Point_("int")(50,229))
trk_ctor.push_back(ROOT.cv.Point_("int")(29,229))

trk_ctor.push_back(trk_ctor[0])
for pt in trk_ctor:
    pt_x.append(pt.x)
    pt_y.append(pt.y)
    plt.plot(pt_x,pt_y,'o-',color='black',lw=1)

ax=plt.gca()
#ax.set_xlim(0,450)
#ax.set_ylim(0,450)
plt.grid()
plt.show()

# make a path representing this contour
path_list1 = []
for pt in trk_ctor:
    path_list1.append((pt.x,pt.y)) 
    
# make a fake shower image...
b=np.zeros((485,485))

shr_ctor = ROOT.std.vector(ROOT.cv.Point_('int'))()

shr_ctor.clear()
    
pt_x=[]
pt_y=[]

shr_ctor.push_back(ROOT.cv.Point_("int")(25,225))
shr_ctor.push_back(ROOT.cv.Point_("int")(50,225))
shr_ctor.push_back(ROOT.cv.Point_("int")(50,229))
shr_ctor.push_back(ROOT.cv.Point_("int")(29,229))

shr_ctor.push_back(shr_ctor[0])
for pt in shr_ctor:
    pt_x.append(pt.x)
    pt_y.append(pt.y)
    plt.plot(pt_x,pt_y,'o-',color='black',lw=1)

ax=plt.gca()
#ax.set_xlim(0,450)
#ax.set_ylim(0,450)
plt.grid()
plt.show()

# make a path representing this contour
path_list2 = []
for pt in shr_ctor:
    path_list2.append((pt.x,pt.y)) 


In [ ]:
path1 = matplotlib.path.Path(path_list1)
path2 = matplotlib.path.Path(path_list2)

img1 = path1.contains_points(np.row_stack(np.where(a==0)).T).reshape(485,485).astype(np.uint8)
img4 = path2.contains_points(np.row_stack(np.where(b==0)).T).reshape(485,485).astype(np.uint8)

###################TRACK IMAGE
img1=np.where(img1>0,img1+80,0)
img2 = img1.copy()
img3 = img1.copy()

M = cv2.getRotationMatrix2D((485/2,485/2),20,1)
#rotate 1
img1 = cv2.warpAffine(img1,M,(485,485))

#get nonzero for 1, and move it...
nzero=np.row_stack(np.where(img1>0)).T
img1[nzero[:,0],nzero[:,1]]=0
img1[nzero[:,0]+20,nzero[:,1]+75]=80

M = cv2.getRotationMatrix2D((485/2,485/2),45,1)
#rotate 2
img2 = cv2.warpAffine(img2,M,(485,485))

#flip 2
img2 = img2[:,::-1]

#get nonzero for 2, and move it...
nzero=np.row_stack(np.where(img2>0)).T
img2[nzero[:,0],nzero[:,1]]=0
img2[nzero[:,0]+10,nzero[:,1]-160]=80

#rotate 3
M = cv2.getRotationMatrix2D((485/2,485/2),45,1)
img3 = cv2.warpAffine(img3,M,(485,485))

#get nonzero for 3, and move it...
nzero=np.row_stack(np.where(img3>0)).T
img3[nzero[:,0],nzero[:,1]]=0
img3[nzero[:,0]+42,nzero[:,1]+168]=80

###################SHOWER IMAGE
img4=np.where(img4>0,img4+80,0)
img5 = img4.copy()
img6 = img4.copy()

#move 4
nzero=np.row_stack(np.where(img4>0)).T
img4[nzero[:,0],nzero[:,1]]=0
img4[nzero[:,0]+13,nzero[:,1]+3]=200


#rotate 5
M = cv2.getRotationMatrix2D((485/2,485/2),20,1)
img5 = cv2.warpAffine(img5,M,(485,485))
#move 5
nzero=np.row_stack(np.where(img5>0)).T
img5[nzero[:,0],nzero[:,1]]=0
img5[nzero[:,0]+43,nzero[:,1]+87]=200

#rotate 6
M = cv2.getRotationMatrix2D((485/2,485/2),45,1)
img6 = cv2.warpAffine(img6,M,(485,485))
#move 6
nzero=np.row_stack(np.where(img6>0)).T
img6[nzero[:,0],nzero[:,1]]=0
img6[nzero[:,0]+23,nzero[:,1]+150]=200

#add them...
trk_img = img1+img2+img3
shr_img = img4+img5+img6
adc_img = trk_img+shr_img

plt.imshow(trk_img+shr_img,cmap='Greys',interpolation='none')
ax=plt.gca()
ax.vlines(227,0,485,color='orange',lw=2,alpha=0.5)
ax.text(50,50,"Fake Image",fontsize=20,fontweight='bold',color='red')
fig=plt.gcf()
fig.set_size_inches(20,12)

plt.rc('grid', linestyle="-", color='black')

mjx = mjy = 10.;
mix = miy = 1.

major_ticks_x = np.arange(0,485+mjx/2,mjx)                                                                                                                                                                                    
major_ticks_y = np.arange(0,485+mjy/2,mjy)                                                                                                                                                                                    
minor_ticks_x = np.arange(0+mix/2,485+mix*1.5,1)                                                                                                                                                                            
minor_ticks_y = np.arange(0+miy/2,485+miy*1.5,1)                                                                                                                                                                            

ax.set_xticks(major_ticks_x)                                                                                                                                                                                            
#ax.set_xticks(minor_ticks_x, minor=True)                                                                                                                                                                                
ax.set_yticks(major_ticks_y)                                                                                                                                                                                            
#ax.set_yticks(minor_ticks_y, minor=True)                                                                                                                                                                                
ax.grid(which='minor')                                                                                                                                                                                                  
ax.set_xticklabels(major_ticks_x.astype(np.uint16),rotation=45,fontsize=10,minor=False)                                                                                                                                                   
ax.set_yticklabels(major_ticks_y.astype(np.uint16),rotation=45,fontsize=10,minor=False) 

plt.grid()
plt.show()

In [ ]:
adc_mat  = pdraw.mat(adc_img.T.astype(np.float32))
track_mat  = pdraw.mat(trk_img.T.astype(np.float32))
shower_mat = pdraw.mat(shr_img.T.astype(np.float32))

In [ ]:
pprocessor.PreProcess(adc_mat,track_mat,shower_mat)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True,figsize=(10,5))
ax1.imshow(adc_img.T,cmap='Greys',interpolation='none')
ax2.imshow(pdraw.image(adc_mat),cmap='Greys',interpolation='none')
ax1.set_title("ADC Before")
ax2.set_title("ADC After")
plt.show()
fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True,figsize=(10,5))
ax1.imshow(trk_img.T,cmap='Greys',interpolation='none')
ax2.imshow(pdraw.image(track_mat),cmap='Greys',interpolation='none')
ax1.set_title("Track Before")
ax2.set_title("Track After")
plt.show()
fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True,figsize=(10,5))
ax1.imshow(shr_img.T,cmap='Greys',interpolation='none')
ax2.imshow(pdraw.image(shower_mat),cmap='Greys',interpolation='none')
ax1.set_title("Shower Before")
ax2.set_title("Shower After")
plt.show()